# Training of a super simple model for celltype classification

In [1]:
import tensorflow as tf
!which python
!python --version
print(tf.VERSION)
print(tf.keras.__version__)
!pwd #  start jupyter under notebooks/ for correct relative paths

/home/phineas/anaconda3/envs/depiction-env/bin/python
Python 3.6.5 :: Anaconda, Inc.
1.14.0
2.2.4-tf
/home/phineas/Documents/repos/dl-interpretability-compbio/notebooks


In [3]:
import datetime
import inspect
import pandas as pd
import numpy as np
import seaborn as sns
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from depiction.models.celltype.celltype import one_hot_encoding, one_hot_decoding

## a look at the data
labels are categories 1-20, here's the associated celltype:

In [4]:
meta_series = pd.read_csv('../data/single-cell/metadata.csv', index_col=0)
meta_series

,cell type name
label,
1,CD11b- Monocyte
2,CD11bhi Monocyte
3,CD11bmid Monocyte
4,Erythroblast
5,HSC
6,Immature B
7,Mature CD38lo B
8,Mature CD38mid B
9,Mature CD4+ T


There are 13 unbalanced classes, and over 80k samples

In [5]:
data_df = pd.read_csv('../data/single-cell/data.csv')
data_df.groupby('category').count()['CD45']

category
1       912
2      6779
3      1278
4     12030
5       261
6       502
7      7796
8       608
9     13964
10     7821
11     3684
12     3025
13     3864
14     6987
15     9564
16      468
17      293
18        5
19      994
20      240
Name: CD45, dtype: int64

In [6]:
data_df.sample(n=10)

,CD45,CD45RA,CD19,CD11b,CD4,CD8,CD34,CD20,CD33,CD123,CD38,CD90,CD3,category
53353,-0.69492,0.881530,-0.114340,-0.087083,1.607600,0.408030,-0.196550,-0.088004,0.31978,-0.127960,-0.11775,-0.006062,-1.25900,11
38224,4.44520,0.476760,-0.077736,-0.016581,3.580100,-0.219880,-0.011004,-0.059784,0.43719,-0.426980,2.31540,0.132890,4.26200,9
33373,4.46960,-0.561500,-0.168880,0.347350,3.076000,1.037300,-0.118600,-0.014868,-0.22350,0.220260,1.83080,1.171200,4.41170,9
11350,-0.31212,-0.095535,-0.161090,-0.105790,-0.010378,-0.025211,-0.040666,-0.073326,1.10760,-0.097798,1.70450,1.056600,2.82610,4
51254,5.25150,1.007700,0.509640,1.050100,-0.056795,5.107800,0.601790,1.621400,0.22450,-0.224580,2.41230,1.014500,5.52270,10
5565,4.72110,1.499900,-0.242840,3.160400,2.883800,-0.343640,0.938980,-0.093330,4.67880,2.689300,4.21780,0.585290,0.59754,2
2398,3.87100,-0.032058,-0.108210,2.530300,0.743860,-0.021683,0.960550,-0.082569,4.30760,1.066600,4.28950,1.248200,0.35031,2
47280,4.37550,0.287050,-0.088491,-0.086775,0.458480,3.260700,-0.129590,-0.054280,-0.27620,0.289830,0.50937,-0.155450,4.02040,10
17804,-0.20823,-0.103150,-0.153510,0.509170,0.465500,-0.000060,1.605400,0.190660,2.03560,-0.147480,2.41240,0.737840,3.04220,4
74743,3.77460,3.173900,-0.239940,0.257330,-0.172100,4.373700,1.692500,0.319660,-0.15248,-0.141140,0.47485,-0.059107,4.27750,15


In [7]:
print(inspect.getsource(one_hot_encoding)) # from keras, but taking care of 1 indexed classes
print(inspect.getsource(one_hot_decoding))

def one_hot_encoding(classes):
    return to_categorical(classes)[:, 1:]  # remove category 0

def one_hot_decoding(labels):
    return labels.argmax(axis=1) + 1



In [8]:
classes = data_df['category'].values
labels = one_hot_encoding(classes)

#scale the data from 0 to 1
min_max_scaler = MinMaxScaler(feature_range=(0, 1), copy=True)
data = min_max_scaler.fit_transform(data_df.drop('category', axis=1).values)
data.shape

(81075, 13)

In [9]:
one_hot_decoding(labels)

array([ 1,  1,  1, ..., 20, 20, 20])

In [10]:
data_train, data_test, labels_train, labels_test = train_test_split(
    data, labels, test_size=0.33, random_state=42, stratify=data_df.category)

In [11]:
labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [12]:
batchsize = 32

In [13]:
dataset = tf.data.Dataset.from_tensor_slices((data_train, labels_train))
dataset = dataset.shuffle(2 * batchsize).batch(batchsize)
dataset = dataset.repeat()

testset = tf.data.Dataset.from_tensor_slices((data_test, labels_test))
testset = testset.batch(batchsize)

## I don't know how a simpler network would look like

In [14]:
model = tf.keras.Sequential()
# Add a softmax layer with output units per celltype:
model.add(layers.Dense(
    len(meta_series), activation='softmax',
    batch_input_shape=tf.data.get_output_shapes(dataset)[0]
))

W1103 19:54:27.627181 139803615647552 deprecation.py:506] From /home/phineas/anaconda3/envs/depiction-env/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                280       
Total params: 280
Trainable params: 280
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.categorical_accuracy])

In [17]:
# evaluation on testset on every epoch
# log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.fit(
    dataset,
    epochs=20, steps_per_epoch=np.ceil(data_train.shape[0]/batchsize),
    validation_data=testset, #  callbacks=[tensorboard_callback]
)

Epoch 1/20
1698/1698 [==============================] - 2s 1ms/step - loss: 2.1077 - categorical_accuracy: 0.4552 - val_loss: 1.6879 - val_categorical_accuracy: 0.6672
Epoch 2/20
1698/1698 [==============================] - 2s 938us/step - loss: 1.4345 - categorical_accuracy: 0.6918 - val_loss: 1.2277 - val_categorical_accuracy: 0.7554
Epoch 3/20
1698/1698 [==============================] - 2s 925us/step - loss: 1.0864 - categorical_accuracy: 0.7777 - val_loss: 0.9677 - val_categorical_accuracy: 0.8128
Epoch 4/20
1698/1698 [==============================] - 2s 912us/step - loss: 0.8806 - categorical_accuracy: 0.8177 - val_loss: 0.8053 - val_categorical_accuracy: 0.8335
Epoch 5/20
1698/1698 [==============================] - 2s 936us/step - loss: 0.7464 - categorical_accuracy: 0.8369 - val_loss: 0.6948 - val_categorical_accuracy: 0.8455
Epoch 6/20
1698/1698 [==============================] - 2s 952us/step - loss: 0.6522 - categorical_accuracy: 0.8503 - val_loss: 0.6145 - val_categorical

## Is such a simple model interpretable?

In [18]:
# Save entire model to a HDF5 file
model.save('./celltype_model.h5')

In [19]:
# tensorboard --logdir logs/fit

In [20]:
# To recreate the exact same model, including weights and optimizer.
# model = tf.keras.models.load_model('../data/models/celltype_dnn_model.h5')

# What is the effect of increasing model complexity? 
Play around by adding some layers, train and save the model under some name to use with the other notebook.

![title](https://i.kym-cdn.com/photos/images/newsfeed/000/531/557/a88.jpg)

In [21]:
model = tf.keras.Sequential()
# Adds a densely-connected layers with 64 units to the model:
model.add(layers.Dense(64, activation='relu', batch_input_shape=tf.data.get_output_shapes(dataset)[0])) # 
# ...
# do whatever you want
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dropout(0.5))
# ...
# Add a softmax layer with output units per celltype:
model.add(layers.Dense(len(meta_series), activation='softmax'))